In [1]:
# total : 58 problem and solutions

In [2]:
# env : pixlake
# we focuing on pyspark dataframe processing
# documentation https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame
%load_ext autoreload
%autoreload 2

In [3]:
# make you auto compeletion faster
# https://stackoverflow.com/questions/40536560/ipython-and-jupyter-autocomplete-not-working
%config Completer.use_jedi = False

In [4]:
import os
import sys

def get_workstation_spark_path(where_are_you : str) -> str:
    if where_are_you == 'titan':
        return '/home/data/ryanchao2012/lib'
    elif where_are_you == 'thor':
        return '/opt/spark/versions'
    else:
        raise ValueError("wrong work station name")

spark_path = get_workstation_spark_path('thor')

print('You have pyspark version : ', os.listdir(spark_path))
os.environ['PYSPARK_PYTHON'] = sys.executable
# spark-2.3, spark-2.4
# os.environ['SPARK_HOME'] = os.path.join(spark_path,'spark-2.3')

# use spakr 3.1

os.environ['SPARK_HOME'] = '/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

You have pyspark version :  ['spark-2.3', 'spark-3.1.2-bin-hadoop2.7', 'spark-3.0', 'spark-3.0.1-bin-hadoop2.7', 'spark-2.3.4-bin-hadoop2.7', 'spark-2.4.7-bin-hadoop2.7', 'spark-2.4', 'spark-3.1']


In [5]:
os.environ['SPARK_HOME']

'/home/joetsai/download/spark-3.1.2-bin-hadoop3.2'

In [6]:
from os.path import join
import pandas as pd
from pyspark.sql import SparkSession as Session
from pyspark.sql import DataFrame
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W, types as T
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
C = F.col

In [7]:
conf = (Conf()
    .set('spark.sql.sources.partitionOverwriteMode', 'dynamic')
    .set('spark.driver.memory', '4g')
    .set('spark.driver.maxResultSize', '1g')
   )

In [8]:
spark = (Session
     .builder
     .appName('pyspark-challenge')
     .master('local[2]')
     .config(conf=conf)
     .getOrCreate())

21/10/05 00:39:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
spark

# Creating DataFrame 

In [10]:
# Copy a dataframe

# https://stackoverflow.com/questions/52287553/how-to-create-a-copy-of-a-dataframe-in-pyspark

x = spark.createDataFrame([[1,2], [3,4]], ['a', 'b'])

x2 = x.alias('x2')

id(x) != id(x2)

True

# Export DataFrame

In [18]:
# 1 hive partitioning
# https://sparkbyexamples.com/apache-hive/hive-partitions-explained-with-examples/

# split the parger table into serval smaller parts based on 
# one or multiple columns (partitioning key, for example, date, state, ...)

# similar to table partitioning available in SQL server or any other RDBMS database


# example

# article_meta = spark.sparkContext.parallelize(
#     [
#         ("美味食記","20160301", "title_1"), 
#         ("國內旅遊","20160301", "title_2"), 
#         ("美味食記","20160302", "title_3"), 
#     ]
# )
# article_meta_sdf = spark.createDataFrame(
#     article_meta,
#     ['category','date','title']
# )

# article_meta_sdf.write.parquet(
#     'output/article_meta.parquet',
#     partitionBy=['category','date']
# )

print(
    os.listdir('output/article_meta.parquet'),
    os.listdir('output/article_meta.parquet/category=美味食記'),
    os.listdir('output/article_meta.parquet/category=國內旅遊'),
    os.listdir('output/article_meta.parquet/category=美味食記/date=20160302'),
    sep='\n\n'
)

# Read Case I
(
    spark.read.parquet('output/article_meta.parquet')
    .where(C("date") == '20160301')
).show()

# Read Case II

(
    spark.read.parquet('output/article_meta.parquet')
    .where(C("category") == '美味食記')
).show()

# Read Case III
(
    spark.read.parquet('output/article_meta.parquet')
    .where(C("category") == '美味食記')
    .where(C("date") == '20160301')
).show()


# pros

# Fast accessed to the data
# Provides the ability to perform an operation on a smaller dataset



['_SUCCESS', 'category=美味食記', '._SUCCESS.crc', 'category=國內旅遊']

['date=20160302', 'date=20160301']

['date=20160301']

['.part-00001-aa7530fb-b7d4-4bb8-9d78-fb931a5bc3af.c000.snappy.parquet.crc', 'part-00001-aa7530fb-b7d4-4bb8-9d78-fb931a5bc3af.c000.snappy.parquet']
+-------+--------+--------+
|  title|category|    date|
+-------+--------+--------+
|title_1|美味食記|20160301|
|title_2|國內旅遊|20160301|
+-------+--------+--------+

+-------+--------+--------+
|  title|category|    date|
+-------+--------+--------+
|title_3|美味食記|20160302|
|title_1|美味食記|20160301|
+-------+--------+--------+

+-------+--------+--------+
|  title|category|    date|
+-------+--------+--------+
|title_1|美味食記|20160301|
+-------+--------+--------+

